In [ ]:
import pandas as pd
import io
import getpass
import msoffcrypto

In [ ]:
registry_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/pupillometry_sah/data/aSAH_DATA_2009_2023_24122023.xlsx'

In [ ]:
password = getpass.getpass()

In [ ]:
decrypted_workbook = io.BytesIO()
with open(registry_data_path, 'rb') as file:
    office_file = msoffcrypto.OfficeFile(file)
    office_file.load_key(password=password)
    office_file.decrypt(decrypted_workbook)

In [ ]:
registry_df = pd.read_excel(decrypted_workbook, sheet_name='DATA')

In [ ]:
registry_df.head()

In [ ]:
print('Number of patients in registry: {}'.format(registry_df['SOS-CENTER-YEAR-NO.'].nunique()))

In [ ]:
registry_df.DCI_YN.value_counts()

Restrict to after 2019 (when we started collecting pupillometry data)

In [ ]:
registry_after_2019_df = registry_df[registry_df['Date_admission'] > '2019-01-01']


In [ ]:
print('Number of patients in registry after 2019: {}'.format(registry_after_2019_df['SOS-CENTER-YEAR-NO.'].nunique()))

In [ ]:
registry_after_2019_df.DCI_YN.value_counts()